# AWS DataZone의 REST API를 직접 호출하여 공유된 App 리스트 조회
- SMUS 공유된 App 화면의 경우 https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/listings/search 에 attribute:"typeName",value:"BedrockPromptAssetType" 페이로드를 이용해서 도메인 내 모든 BedrockPromptAssetType을 가져옴.

- 그리고, https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/subscriptions 에 owningUserId:"2e2de543-52bd-4110-95c9-a8eda4a20ffb", status:"APPROVED" 페이로드를 이용해서 특정 User(owningUserId)에게 APPROVED된 subscription(사용자가 어떤 Listing을 구독하면, 시스템이 각 구독 요청마다 고유 ID를 생성)을 가져옴

- 두 가지를 조합하면 특정 사용자에게 승인된 BedrockPromptAssetType를 가져올 수 있음.

- 다만, https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/listings/search 의 경우 AWS Python SDK에서 search_listing 메서드를 사용해서 가져 올 수 있으나, https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/subscriptions 이 API의 경우 AWS Python SDK에 특정 User(owningUserId) 기준으로 가져 올 수 있는 메서드를 제공하고 있지 않음.

- 결론적으로 https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/subscriptions API를 조회 후 필터링하여 BedrockPromptAssetType 만 조회하도록 구현

### 먼저, SMUS 도메인 내에 등록된 사용자 이름을 기준으로 UserID 찾기

In [2]:
import boto3

region = "ap-northeast-2"
client = boto3.client("datazone", region_name=region)

domain_id = "dzd-cjvglgj4d43fmg"
user_type = "DATAZONE_IAM_USER"  # DataZone에서 IAM 사용자
search_text = "jh.bae@sk.com"  # ARN에서 사용자 이름 추출 (SK E&S의 경우 사번으로 예:AC135)

response = client.search_user_profiles(
    domainIdentifier=domain_id,
    userType=user_type,
    searchText=search_text
)

items = response.get("items", [])
for user in items:
    print("UserId:", user["id"])
    print("IAM ARN:", user["details"]["iam"]["arn"])
    print("Status:", user["status"])


UserId: 2e2de543-52bd-4110-95c9-a8eda4a20ffb
IAM ARN: arn:aws:iam::533616270150:user/jh.bae@sk.com
Status: ACTIVATED


### 위에서 찾은 UserId 기준으로 해당 UserId에 공유된(승인된) App(BedrockChatAssetType) 조회

In [3]:
import boto3
from requests_aws4auth import AWS4Auth
import requests
import json

region = "ap-northeast-2"
service = "datazone"

session = boto3.Session()
credentials = session.get_credentials().get_frozen_credentials()

auth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    service,
    session_token=credentials.token
)

url = f"https://datazone.{region}.api.aws/v2/domains/dzd-cjvglgj4d43fmg/subscriptions"
params = {
    "owningUserId": user["id"],
    "status": "APPROVED"
}

response = requests.get(url, auth=auth, params=params)

if response.status_code == 200:
    all_data = response.json()
    # BedrockChatAssetType만 필터링
    chat_assets = [
        item for item in all_data.get("items", [])
        if item.get("subscribedListing", {}).get("item", {}).get("assetListing", {}).get("entityType") == "BedrockChatAssetType"
    ]
    print(json.dumps(chat_assets, indent=4))  # 보기 좋게 출력
else:
    print(f"Error {response.status_code}: {response.text}")


[
    {
        "createdAt": 1761634927.483,
        "createdBy": "SYSTEM",
        "domainId": "dzd-cjvglgj4d43fmg",
        "id": "4yb3k4reb1q0pk",
        "pushedSubscription": true,
        "status": "APPROVED",
        "subscribedListing": {
            "description": "",
            "id": "6ik8dtf37ip9co",
            "item": {
                "assetListing": {
                    "entityId": "asbgk8ax1c03a0",
                    "entityRevision": "1",
                    "entityType": "BedrockChatAssetType",
                    "forms": "{\"DZ_Internal_Basic_Form\":{\"name\":\"dataassettestagent\",\"description\":\"\"}}",
                    "glossaryTerms": []
                }
            },
            "name": "dataassettestagent",
            "ownerProjectId": "bknisgxzopjuhk",
            "ownerProjectName": "My_Project_mgouoova",
            "revision": "2"
        },
        "subscribedPrincipal": {
            "user": {
                "details": {
                    "s